# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
env = gym.make('SpaceInvadersDeterministic-v4')
#env.render()

In [3]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 6
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [ ]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0


### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        action, value = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        r = reward

        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state, value, 0, 0)
        # Start training after random sample generation
        if(frame % train_frame == 0):
            agent.train_policy_net(frame)
            # Update the target network
            agent.update_target_net()
        score += reward
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 400 and len(evaluation_reward) > 5:
                torch.save(agent.policy_net, "./save_model/breakout_dqn")
                sys.exit()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size])


episode: 0   score: 120.0   memory length: 699   epsilon: 1.0    steps: 699     evaluation reward: 120.0
episode: 1   score: 210.0   memory length: 1363   epsilon: 1.0    steps: 664     evaluation reward: 165.0
episode: 2   score: 135.0   memory length: 1982   epsilon: 1.0    steps: 619     evaluation reward: 155.0
episode: 3   score: 110.0   memory length: 2372   epsilon: 1.0    steps: 390     evaluation reward: 143.75
episode: 4   score: 50.0   memory length: 2827   epsilon: 1.0    steps: 455     evaluation reward: 125.0
episode: 5   score: 110.0   memory length: 3580   epsilon: 1.0    steps: 753     evaluation reward: 122.5
episode: 6   score: 410.0   memory length: 4681   epsilon: 1.0    steps: 1101     evaluation reward: 163.57142857142858
episode: 7   score: 90.0   memory length: 5104   epsilon: 1.0    steps: 423     evaluation reward: 154.375
episode: 8   score: 305.0   memory length: 6209   epsilon: 1.0    steps: 1105     evaluation reward: 171.11111111111111
episode: 9   score

/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:157: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:158: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:159: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Policy loss: -12.450561. Value loss: 10.459412. Entropy: 1.768124.
Iteration 2
Policy loss: -12.725893. Value loss: 9.640344. Entropy: 1.765771.
Iteration 3
Policy loss: -12.775999. Value loss: 8.559490. Entropy: 1.765218.
episode: 14   score: 125.0   memory length: 10518   epsilon: 1.0    steps: 825     evaluation reward: 159.33333333333334
episode: 15   score: 120.0   memory length: 11157   epsilon: 1.0    steps: 639     evaluation reward: 156.875
episode: 16   score: 135.0   memory length: 11792   epsilon: 1.0    steps: 635     evaluation reward: 155.58823529411765
episode: 17   score: 125.0   memory length: 12430   epsilon: 1.0    steps: 638     evaluation reward: 153.88888888888889
episode: 18   score: 610.0   memory length: 13747   epsilon: 1.0    steps: 1317     evaluation reward: 177.89473684210526
episode: 19   score: 105.0   memory length: 14286   epsilon: 1.0    steps: 539     evaluation reward: 174.25
episode: 20   score: 50.0   memory length: 14802   epsilon: 1.0    steps:

In [ ]:
torch.save(agent.policy_net, "./save_model/breakout_dqn")